### Create NN Model for 02-24 Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import numpy as np
from numpy.linalg import pinv
import os


local file

In [ ]:
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/')
data_02_24 = pd.read_csv('./data/cleaned/data_02_24.csv')
df = data_02_24.copy() # import data

colab

In [4]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/colab/data_02_24.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df = df.drop(columns=['Unnamed: 0', 'Team', 'Def'])

#### Normalize Data

copied from 'exploration/nearest_neighbors.ipynb'

In [6]:
def mean_year(group):
        avg = np.average(group, weights=group.index.map(lambda x: df.loc[x, 'PA']))
        return avg

In [7]:
# weighted std to account for low PA outlier preformances
def calculate_weighted_std(group):
    avg = np.average(group, weights=group.index.map(lambda x: df.loc[x, 'PA']))
    variance = np.average((group - avg)**2, weights=group.index.map(lambda x: df.loc[x, 'PA']))
    return np.sqrt(variance)

mean for each year

In [8]:
# Create empty DataFrame to store results
mean_col_year = pd.DataFrame(index=df['Season'].unique())

# Loop through numeric columns
for col in df.select_dtypes(include=[np.number]).columns:
    if col != 'Season' and col != 'MLBAMID':  # skip meaningless numeric
        yearly_mean = df.groupby('Season')[col].apply(mean_year)
        mean_col_year[col] = yearly_mean

mean_col_year = mean_col_year.reset_index()
mean_col_year = mean_col_year.rename(columns={'index': 'Season'})

mean_col_year.sort_values(by=['Season'])
mean_col_year.head(50)

,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
0,2007,479.323660,29.131879,0.087048,0.165649,0.304928,100.059697,0.215508,3.847416,1.871003,0.237717,0.665527,0.596592,0.886100,0.813455,0.085515,0.253939
1,2009,475.489236,28.871330,0.090529,0.174445,0.301505,99.974603,0.142641,3.682090,1.853189,0.248842,0.664887,0.627009,0.882261,0.810436,0.085482,0.257907
2,2014,453.658304,28.465598,0.077702,0.198467,0.300486,99.992646,0.064191,3.242497,1.824187,0.307677,0.662543,0.663046,0.876915,0.798306,0.093996,0.267385
3,2019,447.323061,27.913035,0.086799,0.223620,0.299295,100.064046,0.093284,4.016924,1.793319,0.316885,0.691812,0.630996,0.853214,0.767166,0.110672,0.271976
4,2020,170.703366,28.026721,0.091721,0.234139,0.291201,100.136435,-0.003950,1.503070,0.695515,0.307861,0.680627,0.615423,0.844059,0.755157,0.113645,0.280891
5,2021,434.346692,28.345488,0.088237,0.225855,0.292583,100.027472,0.076247,3.768059,1.777268,0.314417,0.696508,0.628063,0.850632,0.765565,0.111828,0.273242
6,2018,455.833036,28.065419,0.086321,0.216814,0.297583,100.067295,0.142828,3.543007,1.790976,0.309668,0.679508,0.634281,0.859469,0.774731,0.106147,0.271946
7,2008,467.097081,28.828458,0.088692,0.169905,0.301980,99.982303,0.197216,3.719166,1.831372,0.249494,0.659640,0.624337,0.883456,0.813578,0.085154,0.253608
8,2024,448.020185,27.943522,0.081852,0.225702,0.290938,100.173534,0.053078,3.596287,1.787539,0.319775,0.696517,0.625439,0.859905,0.770404,0.110719,0.273644
9,2012,453.774083,28.526712,0.081452,0.191955,0.298890,100.088430,0.103176,3.615414,1.845739,0.302494,0.653050,0.675014,0.876266,0.802582,0.090579,0.265703


std for each year

In [9]:
# Create empty DataFrame to store results
std_col_year = pd.DataFrame(index=df['Season'].unique())

# Loop through numeric columns
for col in df.select_dtypes(include=[np.number]).columns:
    if col != 'Season' and col != 'MLBAMID':  # skip meaningless numeric
        yearly_std = df.groupby('Season')[col].apply(calculate_weighted_std)
        std_col_year[col] = yearly_std

std_col_year = std_col_year.reset_index()
std_col_year = std_col_year.rename(columns={'index': 'Season'})

std_col_year.sort_values(by=['Season'])
std_col_year.head(45)

,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
0,2007,186.438339,4.367115,0.034388,0.057420,0.038677,28.886765,3.851518,17.061512,2.064698,0.059155,0.059302,0.109721,0.049635,0.062472,0.032731,0.027622
1,2009,180.331321,4.019151,0.032891,0.057032,0.038721,28.042407,3.573888,16.125988,2.058909,0.055755,0.059636,0.102275,0.049860,0.061134,0.031560,0.027306
2,2014,182.754092,3.916344,0.030344,0.063330,0.041191,29.311807,3.396804,14.349276,2.024452,0.056597,0.061227,0.095852,0.050305,0.063619,0.033885,0.029648
3,2019,180.535575,3.667468,0.032196,0.064471,0.042027,30.151685,2.354851,16.002425,1.982361,0.062161,0.060492,0.090361,0.049890,0.060312,0.033093,0.029731
4,2020,64.379969,3.654656,0.039277,0.071616,0.058569,37.796240,1.012108,7.408017,0.869445,0.065219,0.064878,0.100243,0.060459,0.068263,0.036236,0.033228
5,2021,180.319863,3.494339,0.031279,0.064601,0.042804,29.169001,2.178201,14.804431,1.867638,0.059613,0.057509,0.088726,0.053418,0.061547,0.033486,0.029649
6,2018,179.370982,3.717087,0.032228,0.064343,0.040083,28.638997,2.471588,15.268044,1.978929,0.061761,0.058405,0.088076,0.050496,0.060659,0.032994,0.029504
7,2008,186.544015,4.222349,0.033790,0.059741,0.036430,28.291714,3.717989,16.034558,2.136428,0.059225,0.057336,0.103243,0.049905,0.061360,0.031749,0.027017
8,2024,180.852846,3.629216,0.028644,0.062498,0.039092,29.464280,2.674628,15.637422,2.048161,0.057921,0.058322,0.093398,0.049193,0.060854,0.033774,0.029235
9,2012,188.300684,4.000032,0.029774,0.061718,0.041901,29.311607,3.461465,14.618813,2.045586,0.056109,0.058640,0.093778,0.049135,0.062510,0.032466,0.026644


z_scores

In [10]:
# z scores for each stat and player, also creating a new df flow, add back names based on MLBAMID later
z_scores = pd.DataFrame(index=df.index, columns=df.select_dtypes(include=[np.number]).columns)

# scaled for each col
for col in df.select_dtypes(include=[np.number]).columns:
    if col not in ['Season', 'MLBAMID']:  # not needed
        for idx in df.index:
            season = df.loc[idx, 'Season']
            value = df.loc[idx, col]
            mean = mean_col_year.loc[mean_col_year['Season'] == season, col].iloc[0]
            std = std_col_year.loc[std_col_year['Season'] == season, col].iloc[0]
            # scale so that 10 is 1 std away
            z_scores.loc[idx, col] = 100 + ((value - mean) / std * 10)

z_scores['Season'] = df['Season']

In [11]:
# cols to numeric
for col in z_scores.columns:
    if col not in ['Season', 'MLBAMID']:
        z_scores[col] = pd.to_numeric(z_scores[col])

reinsert names

In [12]:
normalized_data = z_scores.copy()
normalized_data.insert(0, 'Name', df['Name'])
normalized_data.head(10)

,Name,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
0,brent clevlen,2007,74.826870,85.958971,74.686219,193.060161,107.344122,43.282571,98.653170,96.363387,89.901996,138.703183,88.950287,71.666459,22.211544,37.187201,169.893575,204.596460
1,brett hayes,2009,74.297907,90.367791,72.475999,127.859623,108.219882,117.931506,99.866315,98.235778,91.580480,170.668602,80.735141,60.421763,105.381033,65.578050,147.469352,199.428878
2,carlos peguero,2014,75.723755,96.257739,107.348483,147.612705,148.436984,92.738037,100.377615,97.711317,91.256604,122.457976,73.452584,41.259125,91.336666,33.462464,156.578380,183.278087
3,seby zavala,2019,75.887131,92.057094,73.040518,181.646207,108.098997,44.146682,99.594613,95.874994,90.245625,129.458257,111.272068,55.708080,34.476706,40.158323,179.134016,179.993300
4,franklin barreto,2020,77.834198,88.981943,76.647649,142.109822,78.737137,59.956124,101.610132,90.946521,86.747334,120.942395,87.572409,55.233155,84.442687,56.517822,144.969583,165.941780
5,domingo santana,2014,76.161502,80.937328,92.701724,191.474452,27.049843,35.300547,100.388715,95.262033,88.346507,104.533169,117.006084,37.346862,46.681858,34.973860,163.064364,174.135337
6,khalil lee,2021,76.910658,84.702434,71.790101,176.836023,78.370523,45.201241,99.649956,94.981084,88.131524,103.173196,80.320382,53.364580,65.561270,54.530410,135.519274,172.156207
7,gift ngoepe,2018,75.646393,99.824005,89.546771,164.461808,67.339068,43.622346,99.348982,95.266649,89.338690,97.997082,71.460200,48.628111,78.321189,62.685967,123.659525,168.376763
8,eliezer alfonzo,2008,75.550163,100.406271,73.751790,132.428726,56.320994,41.119256,98.386518,95.994778,89.755634,150.740042,83.057646,74.748966,56.559446,48.895200,160.671646,180.916565
9,scott kingery,2021,76.966115,96.149521,71.790101,162.804735,65.020621,38.454702,99.634244,94.519822,88.636617,120.647050,120.572055,66.782039,34.360819,43.626577,166.147981,165.237711


correlation with WRC+

In [13]:
corr = normalized_data.corr(numeric_only=True)
corr = corr['wRC+']
print(corr)

Season        0.005616
PA            0.528335
Age           0.034580
BB%           0.390001
K%           -0.310795
BABIP         0.671869
wRC+          1.000000
BsR           0.002394
Off           0.610290
WAR           0.612521
O-Swing%     -0.228714
Z-Swing%      0.003293
O-Contact%    0.124176
Z-Contact%    0.133424
Contact%      0.165336
SwStr%       -0.214951
CSW%         -0.288683
Name: wRC+, dtype: float64


#### NN

function to find the str nearest neighbor based on a found nn

In [14]:
def df_apply(df, indices):
    num_samples = indices.shape[0]
    output_index = df.index[:num_samples]
    rows_data = []
    indices_array = indices.astype(int)

    for i in range(num_samples):
        row_neighbors = {}
        neighbor_positional_indices = indices_array[i]

        for j, neighbor_pos_idx in enumerate(neighbor_positional_indices):
            neighbor_row = df.iloc[neighbor_pos_idx]
            name = neighbor_row['Name']
            season = str(neighbor_row['Season'])
            row_neighbors[f'n{j+1}'] = f"{name},{season}"

        rows_data.append(row_neighbors)

    return pd.DataFrame(rows_data, index=output_index)

diff between target player next year, and nn WRC+

In [15]:
def calculate_wrc_diff(row):
    wrc_current = np.nan
    player_name = None

    try:
        player_info = row['n1'].split(',')
        player_name = player_info[0].strip()
        season = int(player_info[1])

        current_lookup = df[(df['Name'] == player_name) & (df['Season'] == season + 1)]['wRC+']

        if not current_lookup.empty:
            wrc_current = current_lookup.iloc[0]
        else:
            return np.nan
    except (IndexError, ValueError, AttributeError, KeyError):
         return np.nan

    wrc_target_val = np.nan
    for i in range(2, 6):
        neighbor_col = f'n{i}'
        if neighbor_col in row and pd.notna(row[neighbor_col]):
            try:
                neighbor_info = row[neighbor_col].split(',')
                neighbor_name = neighbor_info[0].strip()

                if neighbor_name != player_name:
                    neighbor_season = int(neighbor_info[1])

                    neighbor_lookup = df[(df['Name'] == neighbor_name) & (df['Season'] == neighbor_season + 1)]['wRC+']

                    if not neighbor_lookup.empty:
                        wrc_target_val = neighbor_lookup.iloc[0]
                        break
            except (IndexError, ValueError, AttributeError):
                 continue

    if pd.notna(wrc_current) and pd.notna(wrc_target_val):
        return abs(wrc_target_val - wrc_current)
    else:
        return np.nan

In [16]:
X1 = normalized_data[normalized_data['Season'] < 2024] # filter for eval metric
X1 = X1.reset_index(drop=True)
X1 = X1.drop(columns=['BsR', 'Z-Swing%', 'O-Contact%', 'Z-Contact%']) # filter uneed features, more varience
X = X1.drop(columns=['Name', 'Season']) # same as x1 without names

In [17]:
# I chose mahalanobis distance because of the interconnected nature of the dataset
cov_matrix = np.cov(X, rowvar=False)
inv_cov_matrix = pinv(cov_matrix)
nn = NearestNeighbors(n_neighbors=6, metric='mahalanobis', n_jobs=-1, metric_params={'VI': inv_cov_matrix})
nn.fit(X)

NearestNeighbors(metric='mahalanobis',
                 metric_params={'VI': array([[ 1.84243497e-02, -1.78926502e-03,  4.12358500e-04,
         9.84829102e-04,  1.19216359e-03, -4.60006406e-03,
         1.52175569e-02, -2.46154055e-02, -7.14882969e-04,
         3.29096628e-03,  2.81545661e-03,  1.34747812e-03],
       [-1.78926502e-03,  1.02639979e-02, -8.44388744e-04,
         9.40148791e-04,  5.81517114e-04, -1.32717846e-04,
        -8.12...
         1.39033825e-03, -2.72694750e-03,  4.22753394e-03,
        -1.04986557e-03, -3.54781818e-03, -4.03988008e-02,
         2.18913980e-01,  2.16204509e-01,  2.61750763e-02],
       [ 1.34747812e-03,  1.62015552e-04,  5.53442726e-03,
        -4.76546030e-03, -1.15650078e-03,  2.06231107e-03,
         1.21161256e-03, -1.76846220e-03, -1.05750334e-03,
         3.40142069e-02,  2.61750763e-02,  1.83969674e-02]])},
                 n_jobs=-1, n_neighbors=6)

Nearest Neighbor for Every Player in df

In [ ]:
kNear = pd.DataFrame(columns=['n1', 'n2', 'n3', 'n4', 'n5', 'n6'])
indices_test = nn.kneighbors(X, return_distance=False)
kNear = df_apply(df = X1, indices=indices_test)
for i in range(1, 7):  # for columns n1 through n6
    kNear[f'n{i}'] = kNear[f'n{i}'].astype(str)

c:\Users\dalto\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:704: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)


Loss Value

<small> no col or weight adjustment: 371215.12483655626 </n>
drop cols 'BsR', 'Z-Swing%', 'O-Contact%', 'Z-Contact%': 362640.54290783254
</small>

In [ ]:
# Calculate wrc_diff and handle missing values
kNear['wrc_diff'] = kNear.apply(calculate_wrc_diff, axis=1)

# Calculate sum of wrc_diff, ignoring NaN values
miss = kNear['wrc_diff'].sum(skipna=True)
print(miss)

363196.21522179217


#### loss function | optimization

In [34]:
def loss_fn(weights, normalized_data):

    # begin eval function
    weighted_og = normalized_data[normalized_data['Season'] < 2024] # filter for eval metric
    weighted_og = weighted_og.reset_index(drop=True)
    weighted_og = weighted_og.drop(columns=['BsR', 'Z-Swing%', 'O-Contact%', 'Z-Contact%'])
    weighted_x = weighted_og.drop(columns=['Name', 'Season'])

    for col in weighted_x.columns:
        if col in weights:
            weighted_x[col] = weighted_x[col] * weights[col]

    # need to regularize to prevent negative numbers
    cov_matrix1 = np.cov(weighted_x, rowvar=False)
    epsilon = 1e-6
    regularized_cov = cov_matrix1 + np.eye(cov_matrix1.shape[0]) * epsilon
    inv_cov_matrix1 = pinv(regularized_cov)

    nn_fn = NearestNeighbors(n_neighbors=6, metric='mahalanobis', n_jobs=1, metric_params={'VI': inv_cov_matrix1})
    nn_fn.fit(weighted_x)

    # distances and index
    indices_test_fn = nn_fn.kneighbors(weighted_x, return_distance=False)


    # add data using the mapped indices
    kNear_fn = df_apply(df = weighted_og, indices=indices_test_fn) # pass mapped to avoid out of bounds

    for i in range(1, 7):  # for columns n1 through n6
        kNear_fn[f'n{i}'] = kNear_fn[f'n{i}'].astype(str)

    # add miss
    kNear_fn['wrc_diff'] = kNear_fn.apply(calculate_wrc_diff, axis=1)

    # drop inf or na
    kNear_fn = kNear_fn.dropna(subset=['wrc_diff'])
    kNear_fn['wrc_diff'] = kNear_fn['wrc_diff'].clip(upper=1000)

    # calc miss
    miss_fn = kNear_fn['wrc_diff'].sum(skipna=True)
    print(f"miss of {miss_fn}")
    return miss_fn


cols to opti and weights

In [24]:
cols_opti = X.columns
weights = {}
for col in cols_opti:
    weights[col] = 1
print(weights)

{'PA': 1, 'Age': 1, 'BB%': 1, 'K%': 1, 'BABIP': 1, 'wRC+': 1, 'Off': 1, 'WAR': 1, 'O-Swing%': 1, 'Contact%': 1, 'SwStr%': 1, 'CSW%': 1}


check if weights inti and func works as expected

In [ ]:
loss_value = loss_fn(weights = weights, normalized_data=normalized_data)

scipy :(

In [ ]:
from scipy.optimize import differential_evolution

# loss wrapped for function
def loss_fn_wrapper(x):
    weights_dict_temp = {col: x[i] for i, col in enumerate(cols_opti)}
    return loss_fn(weights=weights_dict_temp, normalized_data=normalized_data)


bounds = [(0.1, 5)] * len(cols_opti)

# Optimize | changed instead of a bfhs alg bc of the complex, non differentable space of bounds
opt = differential_evolution(func=loss_fn_wrapper, strategy="best1exp", bounds=bounds, maxiter=5, popsize=10, disp = True, workers = -1)
print(opt.x)

